In [2]:
import pandas as pd
import numpy as np

In [3]:
tasks_dataframe = pd.read_csv(r"C:\Users\Marina\OneDrive\Desktop\Cell_a.csv")

# 1. Dataset Analysis ( aggiungere IMA)

In [4]:
# Removing rows with CPU usage equal to 0 from the dataset
cpu_usage = tasks_dataframe['CPU'] > 0
tasks_dataframe = tasks_dataframe.drop(tasks_dataframe[~cpu_usage].index)
tasks_dataframe

,Job_ID,Task_ID,Arrival_Time,CPU,Memory
0,375000667413,0,603026300,0.041851,0.001169
1,375000669289,0,606413041,0.024968,0.001179
2,375000670586,0,608994453,0.024176,0.001173
3,375000670588,0,608994466,0.019552,0.001163
4,375000670590,0,609042903,0.028044,0.001179
...,...,...,...,...,...
2329127,400465207745,0,2678935469565,0.004677,0.000067
2329128,400465219425,0,2678943690687,0.000343,0.000004
2329129,400465219425,1,2678943690687,0.000557,0.000004
2329130,400465256347,0,2678955330224,0.002459,0.000050


In [5]:
# Converting the Arrival Time column from microseconds to seconds
tasks_dataframe['Arrival_Time'] = tasks_dataframe['Arrival_Time'] / 1000000

# 2. Algorithms

$$ U_{n} = max{(0, U_{n−1} − T_{n})} + X_{n}, n ≥ 1 $$
where:
- U_0 = 0;
- T_n is the inter-arrival time between arrival n − 1 and n;
- X_n is the service time or the n-th arriving task.

In [8]:
import csv
import itertools

class Task:
    def __init__(self, job_id, task_id, arrival_time, cpu_usage, memory_usage):
        self.job_id = job_id
        self.task_id = task_id
        self.arrival_time = arrival_time
        self.cpu_usage = cpu_usage
        self.memory_usage = memory_usage
        self.completion_time = None
        self.service_time = cpu_usage / 0.1

    def __lt__(self, other):
        # Compare tasks based on remaining service time
        return self.cpu_usage < other.cpu_usage

# Function to simulate baseline (LWL dispatching and FCFS scheduling)
def calculate_completion_time_baseline(tasks, num_servers):
    servers = [[] for _ in range(N)]  # List of servers, each server can have multiple tasks
    unfinished_work = [0] *  N # Unfinished work for each server
    inter_arrival_times = [0] * len(tasks)
    task_completion_times = {}  # Dictionary to store completion time for each task
    service_times = []
    message_loads = 0
    
    for n in range(len(tasks)):
        
        # LWL DISPATCHING 
        available_servers = [i for i, server_tasks in enumerate(servers) if not server_tasks]  # Find servers with no tasks
        if available_servers:
            server_id = available_servers[0]  # Choose the first available server
        else:
            server_id = unfinished_work.index(min(unfinished_work))  # Find the server with the least unfinished work
        
        # FCFS SCEDULING
        # Assign the task to the server
        servers[server_id].append(tasks[n])

        # Calculate completion time for the task
        inter_arrival_time = tasks[n].arrival_time - inter_arrival_times[n - 1]
        inter_arrival_times[n] = tasks[n].arrival_time
        unfinished_work[server_id] = max(0, unfinished_work[server_id] - inter_arrival_time) + tasks[n].service_time
        service_times.append(tasks[n].service_time)
        task_completion_times[(tasks[n].job_id, tasks[n].task_id, tasks[n].arrival_time, tasks[n].cpu_usage)] = tasks[n].arrival_time + unfinished_work[server_id]
    
        message_loads += 1  # Increase the message load for each task assignment

    mean_message_load = message_loads / len(tasks)  # Compute average message load

        
    return task_completion_times, service_times, mean_message_load


import heapq

# Function to simulate LWS dispatching and JSN scheduling
def calculate_completion_time_SJN(tasks, num_servers):
    servers = [[] for _ in range(num_servers)]
    unfinished_work = [0] * num_servers
    inter_arrival_times = [0] * len(tasks)
    task_completion_times = {}
    service_times = []
    message_loads = 0

    # Use a priority queue to order tasks by shortest job next (SJN)
    task_queue = []
    for n in range(len(tasks)):
        heapq.heappush(task_queue, (tasks[n].arrival_time, tasks[n].service_time, tasks[n]))

    while task_queue:
        # Get the next task with the shortest remaining service time
        _, _, task = heapq.heappop(task_queue)

        # LWL DISPATCHING 
        available_servers = [i for i, server_tasks in enumerate(servers) if not server_tasks]  # Find servers with no tasks
        if available_servers:
            server_id = available_servers[0]  # Choose the first available server
        else:
            server_id = unfinished_work.index(min(unfinished_work))  # Find the server with the least unfinished work

        # Assign the task to the server
        servers[server_id].append(task)

        # Calculate completion time for the task
        inter_arrival_time = task.arrival_time - inter_arrival_times[n - 1]
        inter_arrival_times[n] = task.arrival_time
        unfinished_work[server_id] = max(0, unfinished_work[server_id] - inter_arrival_time) + task.service_time
        service_times.append(task.service_time)
        task_completion_times[(task.job_id, task.task_id, task.arrival_time, task.cpu_usage)] = unfinished_work[server_id]
        
        message_loads += 1  # Increase the message load for each task assignment

    mean_message_load = message_loads / len(tasks)  # Compute average message load


    return task_completion_times, service_times, mean_message_load


## Dispatching Algorithm


LWL (Least Work Left) assigns the processing resource to the process or job with the least amount of remaining work, reducing the waiting time for processing and thus the mean job response time.

In LWL scheduling, the system keeps track of the remaining work or workload of each process or job. When a resource becomes available, LWL selects the process or job with the smallest amount of unfinished work and allocates the resource to it. This approach aims to minimize the time required to complete each job and reduce overall response times.

By prioritizing the processes or jobs with less work left, LWL can effectively distribute resources and ensure that jobs are processed more efficiently. This helps to decrease the waiting time for each job, leading to a reduction in the mean job response time.


## Scheduling Algorithm 

Among the two scheduling policies, FCFS (First-Come, First-Served) and SJN (Shortest Job Next), the SJN policy generally tends to minimize the mean job response time compared to FCFS.

Here's an explanation of the two scheduling policies:

1. FCFS (First-Come, First-Served):

- The FCFS algorithm processes requests based on their arrival order, assigning servers to them sequentially.
- Requests are served one after another, without considering their duration or job complexity.
- If a long-duration request is processed before a shorter one, the overall response time for requests can increase because the shorter requests have to wait longer.

2. SJN (Shortest Job Next):

- The SJN algorithm prioritizes requests based on their expected or estimated duration.
- When a new request arrives, the SJN algorithm selects the request with the shortest duration and processes it first.
- This approach reduces the overall response time as shorter requests are served quickly, thus reducing the wait time for other requests.

In general, the SJN policy is considered more efficient than FCFS in minimizing the mean job response time since it prioritizes shorter requests that require less processing time. This helps reduce the overall waiting time and improve the average response time for requests.



In [9]:
# Read the dataset and create Task objects
tasks = []
for index, row in tasks_dataframe.iterrows():
    job_id = int(row[0])
    task_id = int(row[1])
    arrival_time = float(row[2])
    cpu_usage = float(row[3])
    memory_usage = float(row[4])
    
    task = Task(job_id, task_id, arrival_time, cpu_usage, memory_usage)
    tasks.append(task)

        
# Calculate completion time for each task
N = 64

# Simulate baseline (LWL dispatching and FCFS scheduling)
completion_times_baseline, service_times_baseline, mean_message_load_baseline = calculate_completion_time_baseline(tasks, N)

# Simulate LWL dispatching and SJN scheduling
completion_times_SJN, service_times_SJN, mean_message_load_SJN = calculate_completion_time_SJN(tasks, N)


# 3. Metrics

## 3.1 Job response time R

Time elapsing since the arrival of the first arriving task of a job until all tasks belonging to that job
have been fully served. The mean job response time R is obtained by averaging response times of all jobs

In [15]:
job_response_times_baseline = {}
for (job_id, task_id, arrival_time, cpu_usage), completion_time in completion_times_baseline.items():
    if job_id not in job_response_times_baseline:
        job_response_times_baseline[job_id] = completion_time
    else:
        job_response_times_baseline[job_id] += completion_time
        
job_response_time_baseline = sum(job_response_times_baseline.values())
job_mean_response_time_baseline = job_response_time_baseline/len(job_response_times_baseline.values())
print("Job Mean Response Time R of the baseline:" , job_mean_response_time_baseline)


job_response_times_SJN = {}
for (job_id, task_id, arrival_time, cpu_usage), completion_time in completion_times_SJN.items():
    if job_id not in job_response_times_SJN:
        job_response_times_SJN[job_id] = completion_time
    else:
        job_response_times_SJN[job_id] += completion_time
        
job_response_time_SJN = sum(job_response_times_SJN.values())
job_mean_response_time_SJN = job_response_time_SJN/len(job_response_times_SJN.values())
print("Job Mean Response Time R of our algorithms:", job_mean_response_time_SJN)

Job Mean Response Time R of the baseline: 3772276.438344625
Job Mean Response Time R of our algorithms: 80.01622684033926


## 3.2 Job slowdown S

Ratio of response time of the job to the sum of service times of all tasks belonging to the job. The
mean job slowdown S is obtained by averaging slowdown values of all jobs.


In [16]:
job_slowdown_S_baseline = job_response_time_baseline/sum(service_times_baseline)
print("Job Slowdown S of the baseline: ", job_slowdown_S_baseline)

job_slowdown_S_SJN = job_response_time_SJN/sum(service_times_SJN)
print("Job Slowdown S of our algorithms: ", job_slowdown_S_SJN)

Job Slowdown S of the baseline:  47143.89302399442
Job Slowdown S of our algorithms:  1.0


# 3.3 Utilization coefficient of server  k, ρk 

Fraction of time that the server k is busy serving tasks. The overall mean utilization coefficient is ρ = (ρ1 + · · · + ρN)/N.


In [20]:
# DA RIVEDERE, probabilmente non è corretta
total_simulation_time = 31
utilizations_baseline = [service_time / total_simulation_time for service_time in service_times_baseline]
mean_utilization_baseline = sum(utilizations_baseline) / 64
#display(pd.DataFrame(utilizations_baseline))
print("Mean Utilization of the baselien:", mean_utilization_baseline)

print(" ")

total_simulation_time = 31
utilizations_SJN = [service_time / total_simulation_time for service_time in service_times_SJN]
mean_utilization_SJN = sum(utilizations_SJN) / 64
#print(utilizations_SJN)
print("Mean Utilization of our algorithms:", mean_utilization_SJN)

Mean Utilization of the baselien: 46519.07090246972
 
Mean Utilization of our algorithms: 46519.070902469706


# 3.4 Messaging load L

Number of messages exchanged between the dispatcher and servers for a given task dispatching. The
mean message load L is obtained by averaging message load values of all tasks.

In [21]:
print("Mean Message Load of the baselien:", mean_message_load_baseline)
print()
print("Mean Message Load of our algorithms:", mean_message_load_SJN)

Mean Message Load of the baselien: 1.0

Mean Message Load of our algorithms: 1.0


In [23]:
######ESEMPIO######
tasks_2 = [Task(0, 0, 0, 1.5, 0.3),
         Task(0, 1, 2, 2.2, 0.5),
         Task(1, 1, 3, 0.8, 0.7),
         Task(2, 0, 4, 3.0, 0.9)]
            
    
µ = 0.1
N = 2
completion_times, service_times, mean_message_load = calculate_completion_time_baseline(tasks_2, N)

# Print completion time for each task
for (job_id, task_id, arrival_time, cpu_usage), completion_time in completion_times.items():
    print(f"Job ID: {job_id}, Task ID: {task_id}, Arrival Time : {arrival_time}, CPU : {cpu_usage},  Completion Time: {completion_time}")

Job ID: 0, Task ID: 0, Arrival Time : 0, CPU : 1.5,  Completion Time: 15.0
Job ID: 0, Task ID: 1, Arrival Time : 2, CPU : 2.2,  Completion Time: 24.0
Job ID: 1, Task ID: 1, Arrival Time : 3, CPU : 0.8,  Completion Time: 25.0
Job ID: 2, Task ID: 0, Arrival Time : 4, CPU : 3.0,  Completion Time: 55.0


In [24]:
job_response_times = {}
for (job_id, task_id, arrival_time, cpu_usage), completion_time in completion_times.items():
    if job_id not in job_response_times:
        job_response_times[job_id] = completion_time
    else:
        job_response_times[job_id] += completion_time
        
job_response_time = sum(job_response_times.values())
job_mean_response_time = job_response_time/len(job_response_times.values())
job_mean_response_time

39.666666666666664

In [25]:
job_slowdown_S = job_response_time/sum(service_times)
job_slowdown_S

1.5866666666666667